# Questão 2  - Método de Newton-Raphson

<img src="questao2.png" alt="Questao 2" width="500">

### Bibliotecas utilizadas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Definindo o vetor das equações não lineares

As equações do sistema são:
\begin{cases}
f_1(x, y) = x^2 + xy - 10 \\
f_2(x, y) = y + 3xy^2 - 57
\end{cases}

Logo,
\begin{bmatrix}
x^2 + xy - 10 \\
y + 3xy^2 - 57
\end{bmatrix}

In [ ]:
def F(X):
    x, y = X
    f1 = x**2 + x*y - 10
    f2 = y + 3*x*(y**2) - 57
    return np.array([f1, f2])

### Definindo a matriz Jacobiana

Ela é formada pelas derivadas parciais de primeira ordem das equações do sistema        
\begin{cases}
\frac{\partial f_1}{\partial x} = 2x + y \\
\frac{\partial f_1}{\partial y} = x \\
\frac{\partial f_2}{\partial x} = 3y^2 \\
\frac{\partial f_2}{\partial y} = 1 + 6xy
\end{cases}

Logo, a Jacobiana é:
\begin{bmatrix}
2x + y & x \\
3y^2 & 1 + 6xy
\end{bmatrix}

In [ ]:
def J(X):
    x, y = X
    df1dx = 2*x + y
    df1dy = x
    df2dx = 3*(y**2)
    df2dy = 1 + 6*x*y
    return np.array([[df1dx, df1dy],
                     [df2dx, df2dy]])

### Parâmetros iniciais
- Vetor inicial [x0, y0]
- Tolerância
- Número máximo de iterações

In [ ]:
X = np.array([10.0, 10.0])
tol = 1e-5
Nmax = 20

### Definindo a função para resolver os sistemas lineares que usaremos

In [ ]:
def eliminacao_gauss(A, B):
    # Cópias para não alterar originais
    A = [row.copy() for row in A]
    B = [[b] if isinstance(b, (int, float)) else b.copy() for b in B]
    n = len(A)

    # Matriz aumentada
    for i in range(n):
        A[i].append(B[i][0])

    # Eliminação direta
    for i in range(n):
        piv = A[i][i]
        if abs(piv) < 1e-12:
            raise ValueError(f"Pivô zero detectado na linha {i}")
        for j in range(i, n+1):
            A[i][j] /= piv
        for k in range(i+1, n):
            fator = A[k][i]
            for j in range(i, n+1):
                A[k][j] -= fator * A[i][j]

    # Substituição regressiva
    Xsol = [0] * n
    for i in range(n-1, -1, -1):
        Xsol[i] = A[i][n]
        for j in range(i+1, n):
            Xsol[i] -= A[i][j] * Xsol[j]

    return np.array(Xsol)

### Loop iterativo do Método de Newton-Raphson

In [ ]:
iteracoes = [X.copy()]  # armazena os vetores de cada iteração

print(f"| {'Iter':>4} | {'x':>12} | {'y':>12} | {'Δx':>13} | {'Δy':>13} |")
print("=" * 70)

for k in range(Nmax):
    Fx = F(X)
    Jx = J(X)

    delta = eliminacao_gauss(Jx.tolist(), (-Fx).tolist())

    X = X + delta
    iteracoes.append(X.copy())

    print(
        f"| {k+1:4d} | {X[0]:+12.6f} | {X[1]:+12.6f} | {delta[0]:+12.6e} | {delta[1]:+12.6e} |")

    if abs(delta[0]) < tol and abs(delta[1]) < tol:
        print(f"Convergência alcançada na iteração {k+1}")
        break
else:
    print("Número máximo de iterações atingido sem convergência.")

### Resultados finais

In [ ]:
print("\nVetor solução (raízes encontradas):")
print(f"x = {X[0]:.6f}, y = {X[1]:.6f}")

Fx_final = F(X)
print("\nValores das funções em (x, y):")
print(f"f1(x,y) = {Fx_final[0]:.6e}")
print(f"f2(x,y) = {Fx_final[1]:.6e}")

### Gráfico da evolução das iterações

In [ ]:
iteracoes = np.array(iteracoes)
plt.figure(figsize=(8, 5))
plt.plot(iteracoes[:, 0], iteracoes[:, 1], 'o-', label='Iterações (x,y)')
plt.title('Evolução das iterações – Método de Newton-Raphson')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.legend()
plt.show()
